# Modelos

Implementaremos distintos modelos y compararemos su desempeño para trabajar el problema planteado sobre el Banknote authentication dataset. Este dataset contiene 1372 muestras de billetes reales, y cuenta con los siguientes atributos:

*   Variance of Wavelet Transformed image (continuous): Hace referencia a la variación de las intensidades de los pixeles de la imagen de la transformada wavelet.
*   Skewness of Wavelet Transformed image (continuous): Mide la antisimetría de la distribución de los valores obtenidos al aplicar la transformada wavelet en la imagen (coeficientes de wavelet). Si el skewness es igual a cero, entonces es una distribución simétrica.
*   Curtosis of Wavelet Transformed image (continuous): Busca determinar la forma de la distribución de coeficientes wavelet.
*   Entropy of image (continuous): Mide la aleatoriedad de la frecuencia en el contenido de la imagen.

Las etiquetas del dataset están dadas por billetes auténticos representados por 0, y billetes falsos representados por 1.

**(1) Modelo de Regresión lineal**

Para este dataset el objetivo es predecir si un billete es falso o verdadero, por lo tanto si queremos realizar un modelo de regresión lineal podemos abordarlo modificando la variable objetivo. En lugar de predecir si el billete es verdadero o falso podemos transformar la variable objetivo en una variable continua que represente un puntage de seguridad o una probabilidad de que el billete sea genuino. Sin embargo, al transformar el problema podemos afectar la interpretabilidad y la utilidad del modelo. Además de esto, la regresión lineal asume una relación lineal entre las features y la función de target lo cual puede o no ser cierto para este dataset. 

A continuación se muestra la implementación del modelo.

In [14]:
# Emplearemos la librería Pandas ya que esta se especializa en el manejo 
# y análisis de estructuras de datos. Esta librería nos permitirá cargar el dataset
import pandas as pd
import matplotlib.pyplot as plt
# Ahora cargaremos el dataset en un Dataframe que nos permitirá destacar las 
#relaciones entre las variables de los datos.
data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt",
                   header=None)
# Luego asignaremos los nombres de cada columna según el feature correspondiente,
# donde la última columna corresponderá a las etiquetas.
data.columns = ["Variance", "Skewness", "Curtosis", "Entropy", "Class"]

In [19]:
# Asignamos features a X
X = data[["Variance", "Skewness", "Curtosis", "Entropy"]]

# Asignamos la variable objetivo a  y
y = data["Class"]

In [11]:
#Inicialmente realizaremos el modelo empleando las librerías de Sklearn 
import numpy as np
#Ejecutaremos el modelo de regresión usando 'Ridge', el cual es un modelo de 
# regresión que permite hacer regularización
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
#La siguiente librería la empleamos para estandarizar las features
from sklearn.preprocessing import StandardScaler

# Estandarizamos las características
#Modificamos la variable objetivo para realizar la regresión
y_regression = y / (y.max() - y.min())
# Estandarizamos las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Definimos los hiperparámetros
alpha = 0.1  # Parámetro de regularización

# Creamos una instancia del modelo Ridge Regression con regularización 
model = Ridge(alpha=alpha)

# Realizamos validación cruzada para garantizar generalización
cv_scores = cross_val_score(model, X_scaled, y, cv=5, scoring='neg_mean_squared_error')

# Convertimos los MSE (mean squared error) a valores positivos
mse_scores = -cv_scores

# Calculamos el error cuadrado promedio sobre todos los folds
average_mse = np.mean(mse_scores)
print("Average Mean Squared Error:", average_mse)


Average Mean Squared Error: 0.03857964683034848


In [6]:
import numpy as np


# Definimos los hiperparámetros 
learning_rate = 0.01
num_iterations = 1000
regularization_param = 0.01
num_folds = 5 #Aumentar el número de folds brinda una evaluzación más detallada del 
#modelo, pero puede requerir más cómputo

# Inicializamos un arreglo para guardar los errores de mínimos cuadrados
mse_scores = []

# Ejecutamos cross-validation
#El dataset es dividido en el número de folds, donde cada fold es usado una vez
# como conjunto de entrenamiento 
fold_size = len(X) // num_folds
for i in range(num_folds):
    # Dividimos los datos en datos de entrenamiento y de prueba para el fold correspondiente
    X_train = np.concatenate((X[:i * fold_size], X[(i + 1) * fold_size:]), axis=0)
    y_train = np.concatenate((y[:i * fold_size], y[(i + 1) * fold_size:]), axis=0)
    X_test = X[i * fold_size:(i + 1) * fold_size]
    y_test = y[i * fold_size:(i + 1) * fold_size]

    # Inicializamos los pesos 
    num_features = X_train.shape[1]
    weights = np.zeros(num_features)

    # Definimos la función de costo (error de mínimos cuadrados)
    def mean_squared_error(X, y, weights, regularization_param):
        predictions = np.dot(X, weights)
        errors = predictions - y
        mse = np.mean(errors ** 2) + regularization_param * np.sum(weights[1:] ** 2)
        return mse

    # Realizamos gradiente en descenso para optimizar los pesos
    for iteration in range(num_iterations):
        predictions = np.dot(X_train, weights)
        errors = predictions - y_train
        gradient = np.dot(X_train.T, errors) / len(X_train) + 2 * regularization_param * weights
        weights -= learning_rate * gradient

    # Calculamos el error de mínimos cuadrados para el fold actual
    mse = mean_squared_error(X_test, y_test, weights, regularization_param)
    mse_scores.append(mse)

# Calculamos el promedio del error de mínimos cuadrados sobre todos los folds
average_mse = np.mean(mse_scores)
print("Average Mean Squared Error:", average_mse)


Average Mean Squared Error: 0.3348471965402167


**(ii) Regresión logística**

La regresión logística es un algoritmo popular para realizar clasificación binaria, por lo tanto, a diferencia del modelo anterior, puede ser pertinente implementarlo para el dataset que estamos trabajando. Para ajustar los hiperparámetros en estos modelos se suelen usar distintos métodos, tales como Regularization Strength, solver algorithm, class wighting, entre otros. En el caso particular del modelo implementado se tiene que que Aumentar el número de folds brinda una evaluzación más detallada del modelo, pero puede requerir más cómputo.


In [20]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

# Removemos los nombres de las características del dataset
X = X.values
y = y.values
# Definimos los hiperparámetros
learning_rate = 0.01
num_iterations = 1000
num_folds = 5

# Inicializamos un arreglo para guardar las precisiones
accuracy_scores = []

# Realizamos cross-validation
fold_size = len(X) // num_folds
for i in range(num_folds):
    # SDibidimos los datos en para entrenamiento y testeo para el fold correspondiente
    X_train = np.concatenate((X[:i * fold_size], X[(i + 1) * fold_size:]), axis=0)
    y_train = np.concatenate((y[:i * fold_size], y[(i + 1) * fold_size:]), axis=0)
    X_test = X[i * fold_size:(i + 1) * fold_size]
    y_test = y[i * fold_size:(i + 1) * fold_size]

    # Estandarizamos las características
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Definimos el modelo de regresión logística
    class LogisticRegression:
        def __init__(self, learning_rate=0.01, num_iterations=1000):
            self.learning_rate = learning_rate
            self.num_iterations = num_iterations

        def sigmoid(self, z):
            return 1 / (1 + np.exp(-z))

        def fit(self, X, y):
            num_samples, num_features = X.shape
            self.weights = np.zeros(num_features)

            for _ in range(self.num_iterations):
                linear_model = np.dot(X, self.weights)
                y_pred = self.sigmoid(linear_model)
                gradient = np.dot(X.T, (y_pred - y)) / num_samples
                self.weights -= self.learning_rate * gradient

        def predict(self, X):
            linear_model = np.dot(X, self.weights)
            y_pred = self.sigmoid(linear_model)
            y_pred_class = np.where(y_pred >= 0.5, 1, 0)
            return y_pred_class

    # Creamos una instancia del modelo
    model = LogisticRegression(learning_rate=learning_rate, num_iterations=num_iterations)

    # Ajustamos el modelo a los datos de entrenamiento
    model.fit(X_train_scaled, y_train)

    # Realizamos predicciones en los datos de prueba
    y_pred = model.predict(X_test_scaled)

    # Calculamos la precisión para el fold correspondiente
    accuracy = np.mean(y_pred == y_test)
    accuracy_scores.append(accuracy)

# Calculamos el promedio de la precisión sobre todos los folds
average_accuracy = np.mean(accuracy_scores)
print("Average Accuracy:", average_accuracy)


Average Accuracy: 0.9620437956204378


**(iii) Árboles de decisión**

Este es un algoritmo conocido por su efectividad en tareas de clasificación y regresión. Para ajustar los hiperparámetros de este modelo se pueden realizar los mismos métodos que se mencionaron en el caso anterior, adicionalmente existen librerías que ajustan los hiperparámetros automáticamente como por ejemplo ' GridSearchCV' o 'RandomizedSearchCV'.

In [21]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


# Definimos los hiperparámetros
num_folds = 5
max_depth = 3  # Máxima profundidad del árbol de decisión
min_samples_split = 2  # Mínimum número de samples requeridos para dividir
# un nodo interno

# Inicializamos un arreglo para guardar las precisiones
accuracy_scores = []

# Hacemos cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Definimos el clasificador del árbol de decisión con los hiperparámetros especificados
    model = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split)

    # Ajustamos el modelo a los datos de entrenamiento
    model.fit(X_train, y_train)

    # Hacer predicciones sobre los datos de prueba
    y_pred = model.predict(X_test)

    # Calculamos la precisión sobre el foldn actual
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

# Calculamos el promedio de la precisión sobre todos los folds
average_accuracy = np.mean(accuracy_scores)
print("Average Accuracy:", average_accuracy)


Average Accuracy: 0.9271161247511612


**(iv) KNN**

Para ajustar los hiperparámetros en este modelo existen distintos métodos, entre los más comunes están el número de neighbors, el cual entre más grande $k$ hay mayor suavidad de la barrera de decisión; la métrica usada, weighting scheme, feature scaling, reducción de dimensionalidad entre otros.

In [9]:
import numpy as np
from collections import Counter

def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

class KNNClassifier:
    def __init__(self, k=5):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        y_pred = [self._predict_single(x) for x in X]
        return np.array(y_pred)

    def _predict_single(self, x):
        distances = [euclidean_distance(x, x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]


# Inicializamos y ajustamos el clasificador KNN
knn = KNNClassifier(k=5)
knn.fit(X_train, y_train)

# Hacemos predicciones sobre los datos de testeo
y_pred = knn.predict(X_test)

# Calculamos la precisión del modelo
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)


Accuracy: 1.0


De los modelos implementados podemos concluir que el que nos brinda mayor precisión es KNN, además, que existen distintos métodos para ajustar los hiperparámetros y que estos deben ser seleccionados de acuerdo al modelo, el dataset y la capacidad de cómputo.